In [ ]:
import os

from langchain_openai import OpenAI, OpenAIEmbeddings
from pinecone import Pinecone

In [ ]:
%pwd

In [ ]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [ ]:
%pwd

In [ ]:
with open('.env') as f:
    os.environ.update(
        line.strip().split('=') for line in f
)

In [ ]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_INDEX = os.environ.get('PINECONE_INDEX')

In [ ]:
pincone = Pinecone(api_key=PINECONE_API_KEY)
index = pincone.Index(PINECONE_INDEX)

In [ ]:
from langchain_pinecone import Pinecone

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1536)
text_field = "text"

query = "What are the specific features or aspects that users appreciate the most in our application?"

vectorstore = Pinecone(
    index, embeddings, text_field, namespace='ns0'
)

In [ ]:
vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-instruct', openai_api_key=OPENAI_API_KEY)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa_result = qa.invoke(query)

## Summarization with Prompt

Call the `PromptTemplate` module from Langchain to setup the instruction the model to perform summarization.

In [ ]:
from langchain import PromptTemplate

In [ ]:
template = """
%INSTRUCTIONS:
Please summarize the following piece of text.

%TEXT:
{text}
"""

# Create a LangChain prompt template that we can insert values to later
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

Also take the search result from PDF content and Google with SerpAPI. The output from the PDF file and also Google search result will be make the OpenAI know more about the current product that officialy released in February 2023.

In [ ]:
confusing_text = """{from_qna}
""".format(
    from_qna = qa_result['result'].strip()
)

print(confusing_text)

Here are the final prompt that contain instruction and text that need to be summarized.

In [ ]:
summarize_prompt = prompt.format(text=confusing_text)
print(summarize_prompt)

Perform summarization with OpenAI `text-davinci-003` model.

In [ ]:
summerize = OpenAI(temperature=0, model_name='gpt-3.5-turbo-instruct', openai_api_key=OPENAI_API_KEY)
summarize_result = summerize.invoke(summarize_prompt)
print(summarize_result)